In [ ]:
!pip install sentence-transformers
import os
import re
import random
import numpy as np
from collections import defaultdict
from sentence_transformers import SentenceTransformer, InputExample, losses, util, models
from torch.utils.data import DataLoader






In [ ]:
def parse_query_file(file_path):
    queries = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        query_id, query_text = None, None
        for line in f:
            line = line.strip()
            if line.startswith("<num>"):
                query_id = line.replace("<num>", "").replace("</num>", "").strip()
            elif line.startswith("<title>"):
                query_text = ""
            elif line.startswith("</title>") and query_id and query_text:
                queries[query_id] = query_text.strip()
            elif query_id is not None and query_text is not None:
                query_text += " " + line.strip()
    return queries

def parse_trec_corpus(file_path):
    items = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    doc_id, content, inside = None, [], False
    for line in lines:
        line = line.strip()
        if line == "<DOC>":
            inside = True
            doc_id, content = None, []
        elif line.startswith("<DOCNO>") and inside:
            doc_id = line.replace("<DOCNO>", "").replace("</DOCNO>", "").strip()
        elif line == "</DOC>" and inside:
            items[doc_id] = " ".join(content).strip()
            inside = False
        elif inside and not line.startswith("<"):
            content.append(line)
    return items

def parse_qrels(file_path):
    qrels = defaultdict(set)
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 4:
                continue
            qid, _, docid, rel = parts
            if rel == '1':
                qrels[qid].add(docid)
    return qrels




In [ ]:
def normalize_code_mixed(text):
    norm_map = {
        "valo": "bhalo", "paira": "pigeon", "oxygen cylinder": "oxygen tank",
        "doctor": "dr", "ilish": "hilsa", "sobuj": "green"
    }
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    for k, v in norm_map.items():
        text = text.replace(k, v)
    return re.sub(r'\s+', ' ', text).strip()


query_path = "/content/Train_query_20_Roman.trec"
corpus_path = "/content/Baseline_Corpus.trec"
qrel_path = "/content/QRels_Train_20.txt"

queries = parse_query_file(query_path)
documents = parse_trec_corpus(corpus_path)
qrels = parse_qrels(qrel_path)

print(f" Loaded {len(queries)} queries and {len(documents)} documents.")


train_examples = []

for qid, query_text in queries.items():
    query = normalize_code_mixed(query_text)
    relevant_docs = qrels.get(qid, set())
    all_doc_ids = list(documents.keys())

    for rel_doc in relevant_docs:
        if rel_doc not in documents:
            continue
        pos_doc = normalize_code_mixed(documents[rel_doc])
        negatives = random.sample([d for d in all_doc_ids if d not in relevant_docs], 3)
        for neg_id in negatives:
            neg_doc = normalize_code_mixed(documents[neg_id])
            train_examples.append(InputExample(texts=[query, pos_doc, neg_doc]))

#  Fine-Tune the E5 Model
model = SentenceTransformer('intfloat/multilingual-e5-small')
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.TripletLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=10,
    show_progress_bar=True
)

model.save('./fine-tuned-e5-code-mixed')
print(" Model saved to ./fine-tuned-e5-code-mixed")
model = SentenceTransformer('./fine-tuned-e5-code-mixed')

def rerank(query, top_docs, top_doc_ids, top_k=5):
    query = "query: " + normalize_code_mixed(query)
    docs = ["passage: " + normalize_code_mixed(d) for d in top_docs]
    query_emb = model.encode(query, convert_to_tensor=True)
    doc_embs = model.encode(docs, convert_to_tensor=True)
    sims = util.pytorch_cos_sim(query_emb, doc_embs)[0]
    top_indices = sims.argsort(descending=True)
    return [top_doc_ids[i] for i in top_indices[:top_k]]

In [ ]:
from rank_bm25 import BM25Okapi

# BM25 setup
doc_ids = list(documents.keys())
doc_texts = [normalize_code_mixed(documents[doc_id]) for doc_id in doc_ids]
bm25 = BM25Okapi([doc.split() for doc in doc_texts])
text_to_id = {normalize_code_mixed(documents[doc_id]): doc_id for doc_id in documents}

# Metrics helpers
def ndcg(relevance_list):
    dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance_list)])
    ideal_dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(sorted(relevance_list, reverse=True))])
    return dcg / ideal_dcg if ideal_dcg > 0 else 0

def recall(relevance_list, total_relevant):
    return sum(relevance_list) / total_relevant if total_relevant else 0

# Evaluation function
def retrieve_and_evaluate(queries, qrels, top_k=5, rerank_k=1000):
    precision_scores, map_scores, mrr_scores = [], [], []
    ndcg_scores, recall_scores = [], []

    for qid, query_text in queries.items():
        query_norm = normalize_code_mixed(query_text)
        tokens = query_norm.split()

        # BM25 retrieval
        top_docs = bm25.get_top_n(tokens, doc_texts, n=rerank_k)
        top_doc_ids = [text_to_id[d] for d in top_docs if d in text_to_id]

        # Rerank using E5
        final_doc_ids = rerank(query_text, top_docs, top_doc_ids, top_k=top_k)

        # Ground truth relevance
        relevant = qrels.get(qid, set())
        binary_rel = [1 if doc_id in relevant else 0 for doc_id in final_doc_ids]

        # Precision@k
        precision_scores.append(sum(binary_rel) / top_k)

        # MAP & MRR
        if any(binary_rel):
            ranks = [i + 1 for i, r in enumerate(binary_rel) if r]
            mrr_scores.append(1 / ranks[0])
            ap = sum([rel / (i + 1) for i, rel in enumerate(binary_rel) if rel]) / len(relevant)
        else:
            mrr_scores.append(0)
            ap = 0
        map_scores.append(ap)

        # nDCG and Recall
        ndcg_scores.append(ndcg(binary_rel))
        recall_scores.append(recall(binary_rel, len(relevant)))

    # Results
    print(f"\n Evaluation (Top-{top_k}, Rerank@{rerank_k}):")
    print(f" Precision@{top_k}: {np.mean(precision_scores):.4f}")
    print(f"MAP:              {np.mean(map_scores):.4f}")
    print(f" MRR:              {np.mean(mrr_scores):.4f}")
    print(f" nDCG@{top_k}:      {np.mean(ndcg_scores):.4f}")
    print(f" Recall@{top_k}:    {np.mean(recall_scores):.4f}")


In [ ]:
model = SentenceTransformer('./fine-tuned-e5-code-mixed')
retrieve_and_evaluate(queries, qrels, top_k=5, rerank_k=1000)
